In [3]:
%pip install torch torchvision matplotlib seaborn scikit-learn tqdm

  Using cached torch-2.5.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.20.1-cp311-cp311-win_amd64.whl.metadata (6.2 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/203.1 MB 325.1 kB/s eta 0:10:25
   ---------------------------------------- 0.1/203.1 MB 363.1 kB/s eta 0:09:20
   ----------------

ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    unknown package:
        Expected sha256 603c52d2fe06433c18b747d25f5c333f9c1d58615620578c326d66f258686f9a
             Got        ecade20a0ea96724a216ed654fb4717320b2f09c5abd8434c000f8e88a8f1c12


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   -------------- ------------------------- 73.6/203.1 MB 62.7 kB/s eta 0:34:25
   -------------- ------------------------- 73.6/203.1 MB 61.6 kB/s eta 0:35:02
   -------------- ------------------------- 73.6/203.1 MB 61.6 kB/s eta 0:35:02
   -------------- ------------------------- 73.6/203.1 MB 61.6 kB/s eta 0:35:02
   -------------- ------------------------- 73.6/203.1 MB 61.6 kB/s eta 0:35:02
   -------------- ------------------------- 73.6/203.1 MB 61.6 kB/s eta 0:35:02
   -------------- ------------------------- 73.6/203.1 MB 61.6 kB/s eta 0:35:02
   -------------- ------------------------- 73.6/203.1 MB 61.6 kB/s eta 0:35:02
   -------------- ------------------------- 73.6/203.1 MB 61.6 kB/s eta 0:35:02
   -------------- ------------------------- 73.6/203.1 MB 61.6 kB/s eta 0:35:02
   -------------- ------------------------- 73.6/203.1 MB 61.6 kB/s eta 0:35:02
   -------------- ------------------------- 73.6/203.1 MB 61.6 kB/s eta 0:35:02
   -------------- ---------------------

In [5]:
import os
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split, Subset
from PIL import Image, ImageEnhance
import torch
from ultralytics import YOLO
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    precision_score,
    recall_score,
    f1_score,
    cohen_kappa_score,
)
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [6]:
dataset_directory = "C:/Users/Pratyush/Desktop/BIGDATIOTPROJ/PlantVillage"
output_directory = "C:/Users/Pratyush/Desktop/BIGDATIOTPROJ/PlantVillage/Output"

### Custom loader to handle .JPG files

In [7]:
def is_valid_file(filename: str):
    valid_extensions = {'.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp'}
    return filename.lower().endswith(tuple(valid_extensions))


### Function to apply image augmentation and enhancement

In [8]:
def get_transforms(augment=False):
    transform_list = [
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet-like normalization
    ]
    
    if augment:
        transform_list = [
            transforms.RandomChoice([
                transforms.RandomHorizontalFlip(p=1),
                transforms.RandomVerticalFlip(p=1),
                transforms.RandomApply([transforms.RandomRotation(15)], p=0.5),
                transforms.Compose([transforms.RandomHorizontalFlip(p=1), transforms.RandomVerticalFlip(p=1)])
            ]),
            *transform_list
        ]
    
    return transforms.Compose(transform_list)

### Function to balance the training set

In [9]:
def augment_training_data(subset, target_size):
    base_dataset = subset.dataset
    class_indices = {label: [] for label in range(len(base_dataset.classes))}
    for idx, subset_idx in enumerate(subset.indices):
        _, label = base_dataset[subset_idx]
        class_indices[label].append(subset_idx)
    
    augmented_indices = []
    for label, indices in class_indices.items():
        if len(indices) < target_size:
            extra_count = target_size - len(indices)
            augmented_indices.extend(indices)
            augmented_indices.extend(random.choices(indices, k=extra_count))
        else:
            augmented_indices.extend(indices[:target_size])
    
    return Subset(base_dataset, augmented_indices)

### # Function to create directories and move images to the corresponding folder

In [10]:
def create_and_move_images(subset, base_dir, subset_name):
    # Get the underlying dataset to access the classes
    base_dataset = subset.dataset
    
    # Create directories for train, val, test with class subfolders
    os.makedirs(os.path.join(base_dir, subset_name), exist_ok=True)
    for class_name in base_dataset.classes:
        os.makedirs(os.path.join(base_dir, subset_name, class_name), exist_ok=True)
    
    to_pil_image = transforms.ToPILImage()  # Convert tensor to PIL Image
    
    for idx in range(len(subset)):
        img, label = subset[idx]
        img_name = os.path.basename(subset.dataset.imgs[subset.indices[idx]][0])  # Get the image filename
        class_folder = base_dataset.classes[label]
        
        # Convert the tensor to PIL Image and save it
        pil_img = to_pil_image(img)
        pil_img.save(os.path.join(base_dir, subset_name, class_folder, img_name))

### Function to split dataset and create folders

In [11]:
def split_dataset_and_create_folders(dataset_dir, train_size=0.7, val_size=0.15, test_size=0.15, balance_target_size=1500):
    dataset = datasets.ImageFolder(root=dataset_dir, transform=get_transforms(augment=True), is_valid_file=is_valid_file)
    total_size = len(dataset)
    
    train_len = int(train_size * total_size)
    val_len = int(val_size * total_size)
    test_len = total_size - train_len - val_len
    
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_len, val_len, test_len])
    
    # Augment and balance training data
    balanced_train_dataset = augment_training_data(train_dataset, balance_target_size)
    
    # Apply preprocessing to validation and test sets
    val_dataset.dataset.transform = get_transforms()
    test_dataset.dataset.transform = get_transforms()

    # Create directories and move images into appropriate folders
    create_and_move_images(balanced_train_dataset, dataset_dir, 'train')
    create_and_move_images(val_dataset, dataset_dir, 'val')
    create_and_move_images(test_dataset, dataset_dir, 'test')

    print(f"Data has been split into {os.path.join(dataset_dir, 'train')}, {os.path.join(dataset_dir, 'val')}, {os.path.join(dataset_dir, 'test')}")

### Dataset Preprocessing and Augmentation

In [4]:
dataset_directory = "C:/Users/Pratyush/Desktop/BIGDATIOTPROJ/PlantVillage"
    
split_dataset_and_create_folders(dataset_directory)

NameError: name 'split_dataset_and_create_folders' is not defined

### YOLO11 Training

In [12]:
model = YOLO('yolo11n-cls.pt')

results = model.train(data = dataset_directory, epochs = 10, imgsz = 256, device = 'cpu')

New https://pypi.org/project/ultralytics/8.3.31 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.28  Python-3.12.3 torch-2.3.1+cu118 CPU (AMD Ryzen 7 5800H with Radeon Graphics)
engine\trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=C:/Users/Pratyush/Desktop/BIGDATIOTPROJ/PlantVillage, epochs=10, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mas

train: Scanning C:\Users\Pratyush\Desktop\BIGDATIOTPROJ\PlantVillage\train... 13692 images, 0 corrupt: 100%|██████████| 13692/13692 [00:03<00:00, 3530.87it/s]


train: New cache created: C:\Users\Pratyush\Desktop\BIGDATIOTPROJ\PlantVillage\train.cache


val: Scanning C:\Users\Pratyush\Desktop\BIGDATIOTPROJ\PlantVillage\val... 3095 images, 0 corrupt: 100%|██████████| 3095/3095 [00:00<00:00, 3637.46it/s]


val: New cache created: C:\Users\Pratyush\Desktop\BIGDATIOTPROJ\PlantVillage\val.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 256 train, 256 val
Using 0 dataloader workers
Logging results to runs\classify\train5
Starting training for 10 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

      10/10         0G     0.1775         12        256: 100%|██████████| 856/856 [07:09<00:00,  1.99it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 97/97 [00:33<00:00,  2.91it/s]

                   all      0.953          1



10 epochs completed in 1.342 hours.
Optimizer stripped from runs\classify\train5\weights\last.pt, 3.2MB
Optimizer stripped from runs\classify\train5\weights\best.pt, 3.2MB

Validating runs\classify\train5\weights\best.pt...
Ultralytics 8.3.28  Python-3.12.3 torch-2.3.1+cu118 CPU (AMD Ryzen 7 5800H with Radeon Graphics)
YOLO11n-cls summary (fused): 112 layers, 1,545,239 parameters, 0 gradients, 3.2 GFLOPs
train: C:\Users\Pratyush\Desktop\BIGDATIOTPROJ\PlantVillage\train... found 13692 images in 15 classes  
val: C:\Users\Pratyush\Desktop\BIGDATIOTPROJ\PlantVillage\val... found 3095 images in 15 classes  
test: C:\Users\Pratyush\Desktop\BIGDATIOTPROJ\PlantVillage\test... found 3097 images in 15 classes  


               classes   top1_acc   top5_acc: 100%|██████████| 97/97 [00:29<00:00,  3.24it/s]


                   all      0.953          1
Speed: 0.0ms preprocess, 7.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train5


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Function to split the dataset into train, val, and test if directories don't exist

In [35]:
def ensure_split_dataset(dataset_dir, split_ratios=(0.7, 0.15, 0.15)):
    train_dir = os.path.join(dataset_dir, 'Train')
    val_dir = os.path.join(dataset_dir, 'Val')
    test_dir = os.path.join(dataset_dir, 'Test')

    if not all([os.path.exists(train_dir), os.path.exists(val_dir), os.path.exists(test_dir)]):
        print("Splitting dataset into Train, Val, and Test...")

        # Load the full dataset
        transform = transforms.Compose([
            transforms.Resize((227, 227)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        dataset = datasets.ImageFolder(root=dataset_dir, transform=transform)
        total_len = len(dataset)

        # Calculate sizes for train, val, and test splits
        train_size = int(split_ratios[0] * total_len)
        val_size = int(split_ratios[1] * total_len)
        test_size = total_len - train_size - val_size

        # Split the dataset
        train_dataset, val_dataset, test_dataset = random_split(
            dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42)
        )

        # Create directories and move images into Train, Val, and Test folders
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(val_dir, exist_ok=True)
        os.makedirs(test_dir, exist_ok=True)

        for subset, subset_dir in [(train_dataset, train_dir), (val_dataset, val_dir), (test_dataset, test_dir)]:
            for img_path, label in [dataset.dataset.samples[idx] for idx in subset.indices]:
                class_folder = dataset.classes[label]
                target_folder = os.path.join(subset_dir, class_folder)
                os.makedirs(target_folder, exist_ok=True)
                img_name = os.path.basename(img_path)
                os.rename(img_path, os.path.join(target_folder, img_name))
        
        print("Dataset split completed.")

### Model Preparation and Training (Alexnet)

#### Define transformations for training and testing

In [ ]:
def get_transforms(augment=False):
    transform_list = [
        transforms.Resize((224, 224)),  # AlexNet requires 224x224 images
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet-like normalization
    ]
    
    if augment:
        transform_list = [
            transforms.RandomChoice([
                transforms.RandomHorizontalFlip(p=1),
                transforms.RandomVerticalFlip(p=1),
                transforms.RandomApply([transforms.RandomRotation(15)], p=0.5),
                transforms.Compose([transforms.RandomHorizontalFlip(p=1), transforms.RandomVerticalFlip(p=1)])
            ]),
            *transform_list
        ]
    
    return transforms.Compose(transform_list)

#### Load the dataset

In [74]:
def load_data(dataset_dir, batch_size=32):
    train_dataset = datasets.ImageFolder(root=f'{dataset_dir}/train', transform=get_transforms(augment=True))
    val_dataset = datasets.ImageFolder(root=f'{dataset_dir}/val', transform=get_transforms())
    test_dataset = datasets.ImageFolder(root=f'{dataset_dir}/test', transform=get_transforms())

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, train_dataset.classes

#### Define the AlexNet model with fine-tuning

In [ ]:
def get_alexnet_model(num_classes):
    model = models.alexnet(pretrained=True)
    model.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)
    return model

#### Alexnet Training Function

In [82]:
def train_Alexnet(model, train_loader, val_loader, num_epochs=10, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Ensure the model is on the right device
    if next(model.parameters()).is_cuda:
        print("Model is already on CUDA")
    else:
        model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Debugging: Check for NaN values and label ranges
            if torch.isnan(inputs).any():
                print("NaN values found in inputs!")
            if torch.isnan(labels).any():
                print("NaN values found in labels!")
            
            # Ensure labels are within the correct range
            if (labels >= len(class_names)).any():
                print("Warning: Some labels are out of range!")
            
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            
            running_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)
        
        epoch_accuracy = correct_preds / total_preds
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}, Accuracy: {epoch_accuracy}")
        
        # Validation phase
        model.eval()
        correct_preds = 0
        total_preds = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                correct_preds += (predicted == labels).sum().item()
                total_preds += labels.size(0)
        
        val_accuracy = correct_preds / total_preds
        print(f"Validation Accuracy: {val_accuracy}")
    
    return model

#### Alexnet Evaluation Function

In [83]:
def evaluate_Alexnet(model, test_loader, class_names):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    cm = confusion_matrix(all_labels, all_preds)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()


In [84]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

#### Alexnet Driver Function

In [85]:
dataset_directory = "C:/Users/Pratyush/Desktop/BIGDATIOTPROJ/PlantVillage"
batch_size = 32
num_epochs = 10
    
train_loader, val_loader, test_loader, class_names = load_data(dataset_directory, batch_size)
model = get_alexnet_model(num_classes=len(class_names))
    
# Train the model
model = train_Alexnet(model, train_loader, val_loader, num_epochs=num_epochs)
    
# Evaluate the model and plot confusion matrix
evaluate_Alexnet(model, test_loader, class_names)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
